# Advanced Architectures / Prompting Techniques 

Advanced techniques of applying LLM to usecases include changes to the architecture of a basic LLM and/or unique prompting techniques and often a combination of both. 

(Part 1)
 - Chain of Thought (CoT)
 - ReAct
 - OpenAI Function Calling
 - Prompt Chaining
 
(Part 2)
 - RAG
 - Tree of Thoughts (ToT)
 - Self-Consistency
 - Self-Ask

In [ ]:
import openai
from openai import OpenAI

my_key = "voc-863264702126677398047467ee2c4e76d1d8.74527469"
# my_key = "test"

client = OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key = my_key
)

In [3]:
# LLM function call
def get_response(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=512):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content

## Chain of Thought
**Note**: LLMs dont think and reason as technique name suggests. This technique simply triggers a slightly different response that enables arriving at the right answer. 
 - Tells the model to think step by step before answering. Why? Improves reasoning, math, logic, and complex decision tasks.
 - As simple as, telling it to think in steps. 

In [4]:
system_prompt = "You are a helpful assistant who addresses queries in any topic"
user_prompt = """A farmer needs to cross a river with a fox, a goose, and a bag of beans.
He has a boat, but it can only carry him and one other item at a time.
If left alone together then The fox will eat the goose and The goose will eat the beans.
How can the farmer get all three across safely?
Let's think through each step one by one to ensure nothing is eaten."""

messages=[
          {"role": "system","content": system_prompt},
          {"role": "user","content": user_prompt}
          ]

response = get_response(messages)
print(response)

Sure, let's plan out the steps carefully to ensure that the fox, goose, and bag of beans all cross the river safely.

Step 1: The farmer takes the goose across the river and leaves it on the other side.
Step 2: The farmer goes back to the original side alone, leaving the goose on the other side.
Step 3: The farmer takes the fox across the river.
Step 4: The farmer leaves the fox on the other side and takes the goose back with him to the original side.
Step 5: The farmer leaves the goose on the original side and takes the bag of beans across the river.
Step 6: The farmer leaves the bag of beans with the fox on the other side and goes back to the original side alone.
Step 7: Finally, the farmer takes the goose across the river one last time.

By following these steps, the farmer can successfully transport the fox, goose, and bag of beans across the river without any of them being eaten.


## ReAct Technique
ReAct is using the “reasoning and acting” (ReAct) framework to combine chain of thought (CoT) reasoning with external tool use.
- ReAct can be understood most generally as a machine learning (ML) paradigm to integrate the reasoning and action-taking capabilities of LLMs.
- It has been extended to AI Agents which use this conceptual framework a lot
- ReAct framework uses prompt engineering to structure LLMs activity in a formal pattern of alternating thoughts, actions and observations.
- Reference : https://www.ibm.com/think/topics/react-agent

- Without ReACT Prompting

In [34]:
# Creating the prompt
system_prompt = "You are a helpful Wellness assistant with a goal to improve the wellness of the user"
user_prompt = "How can I know my diet is improving my wellness?"


messages=[
          {"role": "system","content": system_prompt},
          {"role": "user","content": user_prompt}
          ]

response = get_response(messages)
print(response)

There are several ways to determine if your diet is improving your wellness. Here are some tips to help you assess the impact of your diet on your overall well-being:

1. **Energy levels**: Pay attention to your energy levels throughout the day. A well-balanced diet that includes a variety of nutrients should provide you with sustained energy levels and help you feel more alert and focused.

2. **Mood**: Your diet can also have an impact on your mood. Notice if you feel more positive, less irritable, or more emotionally balanced after making changes to your diet.

3. **Digestive health**: A healthy diet should support good digestion and regular bowel movements. If you experience less bloating, gas, or indigestion, it could be a sign that your diet is benefiting your digestive health.

4. **Weight management**: If weight management is a goal for you, tracking your weight and body measurements can help you determine if your diet is supporting your goals.

5. **Physical health**: Improved

- With ReACT Prompting

In [35]:
system_prompt ="""Your goal is to improve the wellness of the user by interleaving thought, action, and observation steps.
              (Thought Step) Begin by assessing the user's current wellness situation. Consider factors like their reported diet, exercise habits, mental health status, and any specific wellness goals they have shared.
              (Action Steps) Collect[Data from user] - Engage with the user to gather essential wellness information, data, or metrics. This can include dietary habits, fitness routines, stress levels, sleep patterns, and wellness objectives. 
                             Provide[Wellness Information] - Based on the collected data and current wellness trends, offer knowledge and insights about nutrition, exercise regimes, mental wellness practices, and relevant biological or medical information that supports and improves wellness. 
                             Recommend[Plan] - Conclude with a tailored recommendation or a specific action plan that the user can implement to enhance their wellness. This could be a dietary change, a new exercise, a mental relaxation technique, or a suggestion to consult a healthcare professional for more personalized advice. 
              (Observation Step) Respond to the user with the Action Steps, and observe the user's response and engagement. Gauge their understanding and willingness to follow the suggestions. Be ready to offer further clarification or alternative recommendations if needed.
              Repeat these steps N times until the user's wellness has improved.
              Example: 
              [User Query] I'm feeling stressed and not sleeping well. What can I do to improve my sleep? 
              (Thought) User is experiencing stress and poor sleep, likely interconnected issues. 
              Collect[Details about user's current stressors and sleep habits], 
              Provide[Information on relaxation techniques and sleep hygiene practices]. 
              Recommend)[Plan] Consider trying meditation before bed and establishing a regular sleep schedule. 
              What are some current stressors in your life? How many hours of sleep do you get each night?
              Have you tried meditation before bed? Do you have a regular sleep schedule?
              Consider trying meditation before bed and establishing a regular sleep schedule.
              Let's create a plan to meditate for 10 minutes before bed each night this week.
              What are some other wellness goals you have or wellness issues you are experiencing?
              """
user_prompt = "How can I know my diet is improving my wellness?"


messages=[
          {"role": "system","content": system_prompt},
          {"role": "user","content": user_prompt}
          ]

response = get_response(messages)
print(response)

(Thought) User is interested in understanding how their diet impacts their overall wellness.

Collect[Details about user's current diet]: What does a typical day of eating look like for you? Do you have any specific dietary preferences or restrictions?

Provide[Wellness Information]: A balanced diet plays a crucial role in improving overall wellness. It should include a variety of fruits, vegetables, whole grains, lean proteins, and healthy fats. Monitoring how your body feels, your energy levels, digestion, and mood can give you insights into how your diet is affecting your wellness.

Recommend[Plan]: To assess if your diet is improving your wellness, consider keeping a food journal to track what you eat and how you feel after eating. Look for patterns in your energy levels, mood, and any physical symptoms. You can also consult with a nutritionist or dietitian for personalized guidance on how to optimize your diet for better wellness outcomes.

Observation: How do you feel after meals

 - Next to do: https://www.ibm.com/think/tutorials/deploy-langgraph-react-agent-manage-it-support-tickets-watsonx-ai

## OpenAI Function Calling
 ReACT Methodology's authors (as in the original paper) suggested that LLM should be able reason and act including making API calls to other programs but there was no such facility until OpenAI added "Function Calling" feature. OpenAI fine-tuned its models such that it can recognize the need to call another function and suggests it to the user. 
 - In a typical Function calling workflow, LLM suggests making a function call by naming a function and providing arguments for the function, user triggers the function call on their backend and returns the results back to the LLM as next input. LLM concludes the worlflow with the final word based on the input received. 
  - Reference: https://openai.com/index/function-calling-and-other-api-updates/
  

In [5]:
# Define the function schema
functions = [
    {
        "name": "get_patient_history",
        "description": "Returns historical health metrics for a patient",
        "parameters": {
            "type": "object",
            "properties": {
                "patient_id": {
                    "type": "string",
                    "description": "The unique ID of the patient"
                }
            },
            "required": ["patient_id"]
        }
    }
]


In [6]:
# Define a dummy function that returns patient previous record (hard-coded below for simplicity)
def get_patient_history(patient_id):
    return {
        "blood_pressure": "150/95",
        "cholesterol": "220 mg/dL",
        "blood_sugar": "140 mg/dL",
        "diagnosis": "Hypertension and pre-diabetes",
        "last_checkup": "2024-11-10"
    }

In [13]:
# Initial/ First message to ChatGpt where we expect it to decide on calling a function
messages = [
    {
        "role": "system",
        "content": """You are a wellness assistant helping users analyze and improve their health.
        If the user asks for comparisons, trends, or historical context, and you do not have that data, use the `get_past_health_metrics` function to retrieve it.
        Always call a function when past health data is relevant but missing."""
    },
    {
        "role": "user",
        "content": """
My patient ID is 001. Here are my current metrics: 
blood pressure is 130/85, cholesterol is 195, and blood sugar is 110.
How am I doing compared to my last checkup?
"""
    }
]

In [14]:
import json
# Step 1: Let LLM decide if it wants to call a function
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,
    function_call="auto"
)

# Step 2: Handle function call from LLM
message = response.choices[0].message

print(message.function_call)

FunctionCall(arguments='{"patient_id":"001"}', name='get_patient_history')


In [15]:
if message.function_call:
    function_name = message.function_call.name
    arguments = json.loads(message.function_call.arguments)
    
    if function_name == "get_patient_history":
        function_response = get_patient_history(arguments["patient_id"])
        
        # Step 3: Add function result to messages and call LLM again
        messages.extend([
            message,
            {
                "role": "function",
                "name": function_name,
                "content": json.dumps(function_response)
            }
        ])
        
        final_response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        
        print(final_response.choices[0].message.content)
    else:
        print("Unrecognized function.")
else:
    print("No function call made.")

Compared to your last checkup on November 10, 2024, there have been improvements in your health metrics. 

Your current blood pressure is 130/85, which is lower than the previous reading of 150/95. Your cholesterol has decreased from 220 mg/dL to 195 mg/dL, showing progress. Additionally, your blood sugar level has improved from 140 mg/dL to 110 mg/dL.

Overall, it appears that your efforts to manage your health have been successful, and you are moving in the right direction. Keep up the good work! If you have any more questions or need further assistance, feel free to ask.


**Note:OpenAI Function Calling**

While OpenAI's Function Calling has a key limitation: the responsibility of writing, maintaining, and updating these functions lies entirely with the developer. This is not scalable since a provider might change their API which will lead to errors in our pipeline. 

Emerging standards like Anthropic’s Model Context Protocol (MCP) and Google’s A2A (Agents-to-Agents) protocol aim to address this by enabling more interoperable and standardized interfaces. 

---

## Prompt Chaining
Prompt Chaining is where we have tasks that require multiple calls to the LLM where the output of one call is fed to the next and this is repeated until task is done. An example use case with translating communications to another language, summarizing and writing a response and converting it back to the orginal language (as below.)


In [16]:
# Step 0: Sample email in French from a job applicant
french_email = """
Bonjour,  
Je m'appelle Claire Dubois et je suis très intéressée par le poste de Data Scientist dans votre entreprise.  
J'ai trois ans d'expérience dans l'analyse de données et le développement de modèles prédictifs.  
Je suis disponible pour un entretien à votre convenance.  
Veuillez trouver mon CV en pièce jointe.  
Cordialement,  
Claire Dubois
"""

In [ ]:
# Step 1: Translate French → English
translate_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": f"Translate this French email into English:\n\n{french_email}"}
    ],
    temperature=0
).choices[0].message.content

In [ ]:
# Step 2: Summarize the English email
summary_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": f"Summarize this email:\n\n{translate_response}"}
    ],
    temperature=0
).choices[0].message.content

In [ ]:
# Step 3: Write a professional reply in English
reply_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": f"Write a professional email reply to this summary:\n\n{summary_response}"}
    ],
    temperature=0.7
).choices[0].message.content

In [22]:
# Step 4: Translate the reply back to French
translated_reply = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": f"Translate this reply into French:\n\n{reply_response}"}
    ],
    temperature=0
).choices[0].message.content

# Final Output
print("=== Step 1: English Translation ===")
print(translate_response)
print("\n=== Step 2: Summary ===")
print(summary_response)
print("\n=== Step 3: English Reply ===")
print(reply_response)
print("\n=== Step 4: Final French Reply ===")
print(translated_reply)

=== Step 1: English Translation ===
Hello,
My name is Claire Dubois and I am very interested in the Data Scientist position in your company.
I have three years of experience in data analysis and developing predictive models.
I am available for an interview at your convenience.
Please find my CV attached.
Best regards,
Claire Dubois

=== Step 2: Summary ===
Claire Dubois is interested in the Data Scientist position at the company, with three years of experience in data analysis and developing predictive models. She is available for an interview and has attached her CV for consideration.

=== Step 3: English Reply ===
Dear Claire,

Thank you for your interest in the Data Scientist position at our company. We appreciate your three years of experience in data analysis and developing predictive models. We have received your CV and will review it for consideration. We will be in touch soon to schedule an interview.

Best regards,

[Your Name]
[Your Title]
[Company Name]

=== Step 4: Final Fr